In [1]:
%%capture
!pip install TTS
!pip install torch
!pip install audiocraft



# Sentiment Analysis

In [2]:
from transformers import pipeline

def generate_sentiment(text,candidate_labels=["happy", "sad", "scary", "adventurous", "calm", "cheerful"]):
  classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device=0) #Run on gpu if available
  result = classifier(text, candidate_labels)

  labels_and_scores = list(zip(result['labels'], result['scores']))
  best_label, best_score = max(labels_and_scores, key=lambda x: x[1])

  return best_label



text = "The dark forest gave me chills."
sentiment_label = generate_sentiment(text)
print(sentiment_label)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

scary


# TTS convert the text to an audiofile

In [3]:
from TTS.api import TTS
from IPython.display import Audio
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
def generate_audio(text,outfile='tts_output.wav'):
  tts_model = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC").to(device)
  tts_model.tts_to_file(text=text,file_path = outfile)
  return outfile

print(device)
text = "The dark forest gave me chills."
outfile = generate_audio(text)
Audio(outfile)




cuda
 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC


 93%|█████████▎| 104M/113M [00:02<00:00, 21.3MiB/s] 

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2



100%|██████████| 113M/113M [00:03<00:00, 33.2MiB/s]


 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample

# Music Generation

In [5]:
from audiocraft.models import MusicGen
import torch
from audiocraft.utils.notebook import display_audio
import soundfile as sf

device = "cuda" if torch.cuda.is_available() else "cpu"
def generate_background_music(label,duration,file_name="background_music.wav"):
  model = MusicGen.get_pretrained('facebook/musicgen-small')
  model.device = device
  model.set_generation_params(
      use_sampling=True,  # Use sampling instead of argmax decoding
      top_k=250,  # Top-K sampling
      duration=duration # TTS Speak time  # Duration of the generated waveform in seconds
  )

  outputs = model.generate(
      descriptions=[
          f'{label} + Background Music'
      ],
      progress=True,
      return_tokens=True
  )
  display_audio(outputs[0], sample_rate=32000)
  audio = outputs[0].cpu().numpy().squeeze()
  sf.write(file_name, audio.T, samplerate=32000)
  print(f"Audio saved to {file_name}")

generate_background_music(sentiment_label, duration=10)




/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Audio saved to background_music.wav


In [6]:
from pydub import AudioSegment

def overlay_tts_on_music(tts_file, music_file, output_file):
    # Load the TTS and music audio files
    tts_audio = AudioSegment.from_wav(tts_file)
    music_audio = AudioSegment.from_wav(music_file)

    tts_duration = len(tts_audio)
    music_duration = len(music_audio)

    if music_duration < tts_duration:
        loops = tts_duration // music_duration + 1
        music_audio = (music_audio * loops)[:tts_duration]
    elif music_duration > tts_duration:
        music_audio = music_audio[:tts_duration]

    combined_audio = music_audio.overlay(tts_audio, position=0)
    combined_audio.export(output_file, format="wav")
    print(f"Overlayed audio saved to {output_file}")

overlay_tts_on_music("tts_output.wav", "background_music.wav", "synced_music.wav")


Overlayed audio saved to synced_music.wav


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
from transformers import pipeline
from audiocraft.models import MusicGen
from pydub import AudioSegment
import soundfile as sf
from TTS.api import TTS
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def generate_sentiment(text, candidate_labels=["happy", "sad", "scary", "adventurous", "calm", "cheerful"]):
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)  # Use GPU if available
    result = classifier(text, candidate_labels)
    return result['labels'][0]

def generate_audio(text, outfile="tts_output.wav"):
    tts_model = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC")
    tts_model.tts_to_file(text=text, file_path=outfile)
    return outfile

def generate_background_music(label, duration, file_name="background_music.wav"):
    model = MusicGen.get_pretrained('facebook/musicgen-small')
    model.device = device
    model.set_generation_params(use_sampling=True, top_k=250, duration=duration)
    outputs = model.generate(descriptions=[f"{label} + Background Music"], progress=True)
    audio = outputs[0].cpu().numpy().squeeze()
    sf.write(file_name, audio.T, samplerate=32000)
    return file_name

def crossfade_music_segments(sentiment_labels, segment_durations, crossfade_duration=2000):
    combined_music = None
    for i, (label, duration) in enumerate(zip(sentiment_labels, segment_durations)):
        music_file = generate_background_music(label, duration)
        music_segment = AudioSegment.from_wav(music_file)

        if combined_music is None:
            combined_music = music_segment
        else:
            combined_music = combined_music.append(music_segment, crossfade=crossfade_duration)

    combined_music.export("combined_music.wav", format="wav")
    return "combined_music.wav"

def overlay_tts_on_music(tts_file, music_file, output_file):
    tts_audio = AudioSegment.from_wav(tts_file)
    music_audio = AudioSegment.from_wav(music_file)

    music_audio = music_audio - 10
    tts_audio = tts_audio

    tts_duration = len(tts_audio)
    music_duration = len(music_audio)

    if music_duration < tts_duration:
        loops = tts_duration // music_duration + 1
        music_audio = (music_audio * loops)[:tts_duration]
    elif music_duration > tts_duration:
        music_audio = music_audio[:tts_duration]

    combined_audio = music_audio.overlay(tts_audio, position=0)
    combined_audio.export(output_file, format="wav")
    print(f"Overlayed audio saved to {output_file}")

def generate_story_audio(story_text, segment_length=500):
    # Split the story into segments
    segments = [story_text[i:i+segment_length] for i in range(0, len(story_text), segment_length)]

    # Analyze sentiment for each segment
    sentiment_labels = [generate_sentiment(segment) for segment in segments]

    # Generate music for each segment and crossfade
    segment_durations = [10] * len(segments)
    combined_music_file = crossfade_music_segments(sentiment_labels, segment_durations)

    # Generate TTS for the entire story
    tts_file = generate_audio(story_text)

    # Overlay TTS on the combined music
    overlay_tts_on_music(tts_file, combined_music_file, "final_story_audio.wav")
    print("Final audio saved to final_story_audio.wav")
import re

def clean_text(text):
    text = text.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")
    text = re.sub(r"\s+", " ", text).strip()
    text = text.replace("—", " - ").replace("–", "-")
    text = re.sub(r"\s+([?.!,])", r"\1", text)
    text = re.sub(r"([?.!,])\s*", r"\1 ", text)
    text = text.strip()

    return text

story_text = """
The great, dark trees of the Big Woods stood all around the house, and beyond them were other trees and beyond them were more trees. As far as a man could go to the north in a day, or a week, or a whole month, there was nothing but woods. There
were no houses. There were no roads. There were no people. There were only trees and the wild animals who had their homes among them. Wolves lived in the Big Woods, and bears, and huge wild cats. Muskrats and mink and
otter lived by the streams. Foxes had dens in the hills and deer roamed everywhere. To the east of the little log house, and to the west, there were miles upon miles of trees,
and only a few little log houses scattered far apart in the edge of the Big Woods. So far as the little girl could see, there was only the one little house where she lived with her father and mother, her sister Mary and baby sister Carrie. A wagon track
ran before the house, turning and twisting out of sight in the woods where the wild animals lived, but the little girl did not know where it went, nor what might be at the end of it
"""
cleaned_text = clean_text(story_text)

generate_story_audio(cleaned_text)


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P